<figure>
  <IMG SRC="https://raw.githubusercontent.com/mbakker7/exploratory_computing_with_python/master/tudelft_logo.png" WIDTH=250 ALIGN="right">
</figure>

# An Introduction to Time Series Analysis with Pastas
*Mark Bakker*

Required files to run this notebook (all available from the `data` subdirectory):
* Head files: `head_nb1.csv`, `B58C0698001_1.csv`, `B50H0026001_1.csv`, `B22C0090001_1.csv`, `head_wellex.csv`
* Pricipitation files: `rain_nb1.csv`, `neerslaggeg_HEIBLOEM-L_967.txt`, `neerslaggeg_ESBEEK_831.txt`, `neerslaggeg_VILSTEREN_342.txt`, `rain_wellex.csv`
* Evaporation files: `evap_nb1.csv`, `etmgeg_380.txt`, `etmgeg_260.txt`, `evap_wellex.csv`
* Well files: `well_wellex.csv`
* Figure: `b58c0698_dino.png`

### Pastas
Pastas is a computer program for hydrological time series analysis and is available from `https://github.com/pastas/pastas`. Pastas makes heavy use of `pandas` `timeseries`. An introduction to `pandas` `timeseries` can be found, for example, [here](http://nbviewer.jupyter.org/github/mbakker7/exploratory_computing_with_python/blob/master/notebook8_pandas/py_exploratory_comp_8_sol.ipynb). The Pastas documentation is available [here](http://pastas.readthedocs.io/en/dev/getting-started.html).

In [ ]:
%matplotlib inline
import pandas as pd
import pastas as ps
import numpy as np
import matplotlib.pyplot as plt

### Load the head observations
The first step in time series analysis is to load a time series of head observations. The time series needs to be stored as a `pandas.Series` object where the index is the date (and time, if desired). `pandas` provides many options to load time series data, depending on the format of the file that contains the time series. In this example, measured heads are stored in the csv file `head_nb1.csv`. 
The heads are read from a csv file with the `read_csv` function of `pandas` and are then squeezed to create a `pandas Series` object.  To check if you have the correct data type, use the `type` command as shown below. 

In [ ]:
ho = pd.read_csv('data/head_nb1.csv', parse_dates=['date'], index_col='date', squeeze=True)
print('The data type of the oseries is:', type(ho))

The variable `ho` is now a `pandas Series` object. To see the first five lines, type `ho.head()`. 

In [ ]:
ho.head()

The series can be plotted as follows

In [ ]:
ho.plot()

In [ ]:
ho.plot(style='.', figsize=(16, 4))
plt.ylabel('Head [m]');
plt.xlabel('Time [years]');

### Load the stresses
The head variation shown above is believed to be caused by two stresses: rainfall and evaporation. Measured rainfall is stored in the file `rain_nb1.csv` and measured potential evaporation is stored in the file `evap_nb1.csv`. 
The rainfall and potential evaporation are loaded and plotted.

In [ ]:
rain = pd.read_csv('data/rain_nb1.csv', parse_dates=['date'], index_col='date', squeeze=True)
print('The data type of the rain series is:', type(rain))

evap = pd.read_csv('data/evap_nb1.csv', parse_dates=['date'], index_col='date', squeeze=True)
print('The data type of the evap series is', type(evap))

plt.figure(figsize=(16, 4))
rain.plot(label='rain')
evap.plot(label='evap')
plt.xlabel('Time [years]')
plt.ylabel('Rainfall/Evaporation (m/d)')
plt.legend(loc='best');

### Recharge
As a first simple model, the recharge is approximated as the measured rainfall minus the measured potential evaporation.

In [ ]:
recharge = rain - evap
plt.figure(figsize=(16, 4))
recharge.plot()
plt.xlabel('Time [years]')
plt.ylabel('Recharge (m/d)');

### First time series model
Once the time series are read from the data files, a time series model can be constructed by going through the following three steps:

1. Creat a `Model` object by passing it the observed head series. Store your model in a variable so that you can use it later on. 
2. Add the stresses that are expected to cause the observed head variation to the model. In this example, this is only the recharge series. For each stess, a `StressModel` object needs to be created. Each `StressModel` object needs three input arguments: the time series of the stress, the response function that is used to simulate the effect of the stress, and a name. In addition, it is recommended to specified the `kind` of series, which is used to perform a number of checks on the series and fix problems when needed. This checking and fixing of problems (for example, what to substitute for a missing value) depends on the kind of series. In this case, the time series of the stress is stored in the variable `recharge`, the Gamma function is used to simulate the response, the series will be called `'recharge'`, and the kind is `prec` which stands for precipitation. One of the other keyword arguments of the `StressModel` class is `up`, which means that a positive stress results in an increase (up) of the head. The default value is `True`, which we use in this case as a positive recharge will result in the heads going up. Each `StressModel` object needs to be stored in a variable, after which it can be added to the model. 
3. When everything is added, the model can be solved. The default option is to minimize the sum of the squares of the errors between the observed and modeled heads. 

In [ ]:
ml = ps.Model(ho)
sm1 = ps.StressModel(recharge, ps.Gamma, name='recharge', settings='prec')
ml.add_stressmodel(sm1)
ml.solve()

The `solve` function has a number of default options that can be specified with keyword arguments. One of these options is that by default a fit report is printed to the screen. The fit report includes a summary of the fitting procedure, the optimal values obtained by the fitting routine, and some basic statistics. The model contains five parameters: the parameters $A$, $n$, and $a$ of the Gamma function used as the response function for the recharge, the parameter $d$, which is a constant base level, and the parameter $\alpha$ of the noise model, which will be explained a little later on in this notebook.
The results of the model are plotted below.

In [ ]:
ml.plot(figsize=(16, 4));

### Response function
The response function of the recharge (which we called `'recharge'`) may be computed and plotted as follows. Note that the effect of recharge on the head dies out after ~600 days. This is called the memory of the system. The exact value used by `pastas` is stored in the `StressModel` object for the recharge, which we called `sm1`. This object has a response function `rfunc`, which has a maximum time `tmax` and a cutoff `cutoff`. The `tmax` correponds to the time when the step function has reached 99% (`cutoff=0.99`) of its final value. 

In [ ]:
h = ml.get_block_response('recharge')
h.plot()
plt.xlabel('time (days)')
plt.ylabel('response to unit recharge (m)')
plt.axhline(0, color='k', linestyle='--')
plt.title(str(sm1.rfunc.cutoff) +  ' cutoff is ' + str(int(sm1.rfunc.tmax)) + ' days');

### Statistics
The `stats` subpackage of `pastas` includes a number of statistical functions that may applied to the model. One of them is the `summary` method, which gives a summary of the main statistics of the model. Use wikipedia if you want to learn about the different statistics. The `stats` package also includes separate functions for all these statistics, as shown below.

In [ ]:
# print(ml.stats.summary())
print('RMSE', ml.stats.rmse())
print('EXPV', ml.stats.evp())

### Evaporation factor
In the previous model, the recharge was estimated as rainfall minus potential evaporation. This is a reasonable first estimate. An improvement is to approximate the actual evaporation as potential evaporation multiplied by a factor, called the evaporation factor. Both the rainfall and evaporation still have the same response function, except that rainfall will cause the head to rise while evaporation will cause the head to drop. 

The previous model may be modified to include estimation of the evaporation factor. Rather than adding a `StressModel` object to the model to simulate the effect of one series (in this case `recharge`), we pass a `StressModel2` object to the model, which takes as input two separate time series, in our case `rain` and `evap`, multiplies the second series by a (negative) factor, but uses the same response function for both stresses. This adds one new parameter to the model: the evaporation factor, which is called `f`.

In [ ]:
ml2 = ps.Model(ho)
sm2 = ps.StressModel2([rain, evap], ps.Gamma, name='rainevap', settings=('prec', 'evap'))
ml2.add_stressmodel(sm2)
ml2.solve()
ml2.plot(figsize=(16, 4));

Note that the new model gives a slightly better fit. The  root mean squared error is lower and the explained variance is higher, but the improvement is not very large. The best-fit evaporation factor (`rainevap_f`) is -1.28, which means that actual evaporation is estimated as 1.28 times potential evaporation.  

In [ ]:
print('rmse old model:', ml.stats.rmse())
print('rmse new model:', ml2.stats.rmse())
print('explained variance old model:', ml.stats.evp())
print('explained variance new model:', ml2.stats.evp())

The separate contributions of rainfall and evaporation can now be plotted below each other as follows

In [ ]:
ml2.plots.decomposition(figsize=(16, 8));

### Exercise 1
Create a model using the head observations, rainfall and potential evaporation of the previous example. Use separate response functions for the rainfall and evaporation and plot the results (you need to correctly specify the `up` keyword argument of the `StressModel` class). Note whether the root mean squared error and explained variance improve. In one graph, plot the block response of the rainfall and the block response of the evaporation. Finally, create a graph showing the contribution of the head variation caused by the rainfall and the contribution caused by the evaporation.

### Noise model

The residuals (the observed minus the modeled head) can be computed and plotted as follows. The root mean squared residuals are added as a title

In [ ]:
r = ml.residuals()
r.plot(figsize=(16,4))
plt.axhline(0, color='r')
plt.ylabel('residuals (m)')
plt.title('Root Mean Squared Residuals: ' + str(round(ml.stats.rmse(), 2)));

It can be seen in the figure shown above that the residuals are not random: there are long periods where the residuals are positive (the observed head is higher than the modeled head) and long periods where the residual is negative (the observed head is lower than the modeled head). This is called autocorrelation. Simply said: the residual of measurement $i$ depends on the residual of measurement $i-1$. This was actually to be expected. One of the reasons for autocorrelated residuals is that the weather station where the rainfall is measured is a certain distance from the observation well. So it is very well possible that rainfall was recorded at the weather station while it didn't rain near the observation well. This means that the model will simulate an increase of the head over a number of days, while the observed head did not increase, resulting in a sequence of negative residuals. The opposite is possible as well, of course, resulting in a sequence of positive residuals.

The problem with autocorrelated residuals is that all estimates of the uncertainty of the parameters of the model are based on the underlying assumption that the residuals are not correlated, in statistical terms: the residuals are  independent. Residuals may be made independent by using a noise model. The simplest noise model is that the residual at measurement $i$ is equal to a factor times the residual at measurement $i-1$ plus a random and independent error $v_i$. The random independent error $v_i$ is also called noise or innovation in time series lingo (the latter is not a very descriptive term, so we will call it noise here). In hydrological time series, the time between observations is rarely constant. It may be expected that the residual of today is stronger correlated with the residual of yesterday than with the residual of last week or last month. The noise model used here is designed such that residual $i$ is equal to a factor times residual $i-1$ plus a random noise, but the factor decreases exponentially when the time between residual $i$ and residual $i-1$ increases. In formula form

$$r_i = \exp(-\Delta t_i / \alpha) r_{i-1} + v_i$$

where $\Delta t_i = t_i - t_{i-1}$ is the time difference between observation $i$ and $i-1$, $v_i$ is the innovation (random error) and $\alpha$ is a parameter that that needs to be fitted. The units of $\alpha$ are time. For example when $\alpha=10$ days, than the residual of today is $\exp(-3)=0.05$ times the residual of 30 days ago (plus a random error, of course). 

A `pastas` model automatically applies a noise model (the `solve` method has the the keyword argument `noise`, which can be set to `False` to turn the noise model off). The noise (innovations) are computed and plotted below. Note that the innovations look much more random than the residuals (there is a statistical check to determine whether they are indeed uncorrelated, but we won't go into that here). 

In [ ]:
v = ml.innovations()
r.plot(figsize=(16,4), label='residuals')
v.plot(figsize=(16,4), label='noise')
plt.axhline(0, color='k');
plt.title('Root Mean Squared Residuals: ' + str(round(ml.stats.rmse(), 2)) + 
          ' Root Mean Squared Noise: ' + str(round(ml.stats.rmsi(), 2)));

### Time series analysis in the Netherlands
The main database for head observation in the Netherlands is called [Dino Loket](https://www.dinoloket.nl). It includes head observations for tens of thousands of observation wells. The observation well we considered thus far has code B58C0698. The data for this file may be downloaded from the Dino Loket (in Dutch) and can be selected on a map or by well code. To use the well code, click on [ondergrondgegevens] and then select [een specifiek object] and then enter the code and select the [grondwatermonitoring] option. On the map, the well is located near the town of Swartbroek (see image below). Head observations in Dino Loket are stored in a very specific format in a csv file. `pastas` includes a routine to read this data. All routines to read data are stored in the `read` subpackage of `pastas`. The routine for reading data from Dino Loket is called `dinodata`. All `read` methods return objects with information found in the data file, including a `series` which contains the observation time series. 

![](data/b58c0698_dino.png)

Weather data for the Netherlands, including rainfall and potential evaporation, may be obtained from the KNMI. It is recommended to select the closest rainfall station and potential evaporation station. Note that there are many more rainfall stations than full weather stations (which include both rainfall and potential evaporation). It is generally more important to have a rainfall station close by than a weather station that records potential evaporation, as the latter varies less from place to place. Data of KNMI rainfall stations can be downloaded from [here](http://projects.knmi.nl/klimatologie/monv/reeksen/select_rr.html). Data of KNMI weather stations can be downloaded from [here](http://projects.knmi.nl/klimatologie/daggegevens/selectie.cgi). KNMI data files can be read by `pastas` using the `read.knmidata` routine. For rainfall stations, rainfall is called `RD`. For weather stations, evaporation is called `EV24`. In the example below, the heads are read from the original Dino Loket files and rainfall and potential evaporation from the original KNMI data sets. As the potential evaporation is a very long record, only the data after 1964 are used. Other than that, the results should be similar to the results of the `ml2` model explained above. 

In [ ]:
ho = ps.read.read_dino('data/B58C0698001_1.csv')
rain = ps.read.read_knmi('data/neerslaggeg_HEIBLOEM-L_967.txt', variables='RD')
evap = ps.read.read_knmi('data/etmgeg_380.txt', variables='EV24')

mlB58 = ps.Model(ho.series)
sm = ps.StressModel2([rain.series, evap.series], ps.Gamma, settings=['prec', 'evap'], name='recharge')
mlB58.add_stressmodel(sm)

mlB58.solve()
mlB58.plot(figsize=(16,4));

### Exercise 2
Consider well B50H0026 located south of the town of Goirle. The closest rainfall station is called Esbeek. The closest weather station for potential evaporation is De Bilt, which has number 260. Create a time series model for this observation well and plot the results.

### Exercise 3
Consider well B22C0090 located in the small town of Witharen. The closest rainfall station is called Vilsteren. The closest weather station for potential evaporation is De Bilt, which has number 260. Load the head observations and note that there is one outlier. Remove the outlier. Create a time series model for this observation well and plot the results. Report the rmse and explained variance for the model with and without the outlier. Make one plot showing the response function with and without the outlier.

### Exercise 4
Create a separate figure that shows the block response for well B58C0698, B50H0026, and B22C009. Which of these wells has the quickest response and which has the slowest response?

### Exercise 5. Effect of pumping
In this exerecise, a time series model is constructed for an observation well where the head variation is caused by  rainfall, evaporation, and a pumping well. 
Head observations are stored in the csv file `head_wellex.csv`. Rainfall is stored in the file `rain_wellex.csv`. Potential evaporation is stored in the file `evap_wellex.csv`. Daily discharge of the pumping well is stored in the file `well_wellex.csv`. All files can be loaded with the `read_csv` function of `pandas`, as shown at the beginning of this notebook. For wells, use the response function `Hantush` rather than `Gamma`. Create two models: 

1. A model with only rainfall and evaporation. Plot the residuals to see that they are very skewed.
2. A model with rainfall, evaporation and the pumping well. Plot the decomposition of the different stresses. What is the approximate drawdown caused by the well?

### Calibration period
The calibration period can be defined with the `tmin` and `tmax` keyword arguments of the `solve` method. The `tmin` and `tmax` can be strings (as in the example below) or `pandas Timestamp` objects. Another keyword argument of the `solve` method is `report`, which can be set to `False` so that no report is printed to the screen. In the example below, we return to the data of the first example. The model is fit on the period from 1995 to 2000 (marked with light blue in the figure). Note that the fit is now much better between 1995 and 2000 as compared to the case when the entire dataset is used (but less good elsewhere).

In [ ]:
# reload the data
ho = pd.read_csv('data/head_nb1.csv', parse_dates=['date'], index_col='date', squeeze=True)
rain = pd.read_csv('data/rain_nb1.csv', parse_dates=['date'], index_col='date', squeeze=True)
evap = pd.read_csv('data/evap_nb1.csv', parse_dates=['date'], index_col='date', squeeze=True)

In [ ]:
ml2 = ps.Model(ho)
sm2 = ps.StressModel2([rain, evap], ps.Gamma, name='rainevap', settings=['prec', 'evap'])
ml2.add_stressmodel(sm2)
ml2.solve(tmin='1995', tmax='2000')
ml2.plot(tmin='1990', tmax='2005', figsize=(16, 4))
plt.axvspan('1995', '2000', alpha=0.8, color='skyblue');

All parameter values are stored in the `parameters` `DataFrame` of a `Model` object

In [ ]:
ml2.parameters

Individual parameters can be accessed by specifying the row and column name using the `.loc['rowname', 'columnname']` syntax, as for any `DataFrame`

In [ ]:
print('optimal evaporation factor:', ml2.parameters.loc['rainevap_f', 'optimal'])

### Exercise 6
A time series is called stationary if the characteristics of the series don't change through time (they are stationary through time). To test whether this is the case, fit model `ml2` as defined above on 7 year periods with starting times every year from 1990 to 2005 (so fit 15 models). Create the following three plots:
1. For each calibration period, plot the simulated head on the same graph (so 15 models on the same graph). First compute the simulated head with `ml2.simulate()` for each model for the same period and store the results, then plot the 15 models on the same graph (using `ml2.plot()` will create 15 separate figures). If there is a lot of spread, the series may not be stationary. You will find that there is one five-year period (starting in 1999) that does a poor job. Apparently, the 5 year periods starting in 1999 is not very representative of the other 5-year periods. 
2. Plot the 15 block response functions of the recharge series of 15 models.
3. Plot the 15 values of the evaporation factor. Does the evaporation factor change much through time? How does that compare to the confidence interval of the evaporation factor?